<a href="https://colab.research.google.com/github/ChristianParsons98/PG_Lab_Code/blob/main/Chem_Mass_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from sympy import symbols, Eq, solve

In [3]:
############################################################
# Created: February 11 2025                                #
# by: Christian Parsons                                    #
# Updated: February 11 2025                                #
# Authors: Christian Parsons                               #
#                                                          #
#                                                          #
# This notebook is used to calculate the mass              #
# of input chemicals for a given output                    #
#                                                          #
############################################################

In [4]:
#Creating a dictionary of the chemical element abreviations as the key and the atomic mass as the value.
atomic_masses = {
    "H": 1.008, "He": 4.0026, "Li": 6.94, "Be": 9.0122, "B": 10.81, "C": 12.011, "N": 14.007, "O": 15.999,
    "F": 18.998, "Ne": 20.180, "Na": 22.990, "Mg": 24.305, "Al": 26.982, "Si": 28.085, "P": 30.974, "S": 32.06,
    "Cl": 35.45, "Ar": 39.948, "K": 39.098, "Ca": 40.078, "Sc": 44.956, "Ti": 47.867, "V": 50.941, "Cr": 51.996,
    "Mn": 54.938, "Fe": 55.845, "Co": 58.933, "Ni": 58.693, "Cu": 63.546, "Zn": 65.38, "Ga": 69.723, "Ge": 72.630,
    "As": 74.922, "Se": 78.971, "Br": 79.904, "Kr": 83.798, "Rb": 85.468, "Sr": 87.62, "Y": 88.906, "Zr": 91.224,
    "Nb": 92.906, "Mo": 95.95, "Tc": 98.0, "Ru": 101.07, "Rh": 102.91, "Pd": 106.42, "Ag": 107.87, "Cd": 112.41,
    "In": 114.82, "Sn": 118.71, "Sb": 121.76, "Te": 127.60, "I": 126.90, "Xe": 131.29, "Cs": 132.91, "Ba": 137.33,
    "La": 138.91, "Ce": 140.12, "Pr": 140.91, "Nd": 144.24, "Pm": 145.0, "Sm": 150.36, "Eu": 151.96, "Gd": 157.25,
    "Tb": 158.93, "Dy": 162.50, "Ho": 164.93, "Er": 167.26, "Tm": 168.93, "Yb": 173.05, "Lu": 174.97, "Hf": 178.49,
    "Ta": 180.95, "W": 183.84, "Re": 186.21, "Os": 190.23, "Ir": 192.22, "Pt": 195.08, "Au": 196.97, "Hg": 200.59,
    "Tl": 204.38, "Pb": 207.2, "Bi": 208.98, "Th": 232.04, "Pa": 231.04, "U": 238.03, "Np": 237.0, "Pu": 244.0,
    "Am": 243.0, "Cm": 247.0, "Bk": 247.0, "Cf": 251.0, "Es": 252.0, "Fm": 257.0, "Md": 258.0, "No": 259.0,
    "Lr": 266.0, "Rf": 267.0, "Db": 270.0, "Sg": 271.0, "Bh": 270.0, "Hs": 277.0, "Mt": 278.0, "Ds": 281.0,
    "Rg": 282.0, "Cn": 285.0, "Nh": 286.0, "Fl": 289.0, "Mc": 290.0, "Lv": 293.0, "Ts": 294.0, "Og": 294.0
}

In [22]:
#Parses a chemical formula into a dictionary of element counts.
#Example: parse_formula("Na3PS4") -> {"Na": 3, "P": 1, "S": 4}

def parse_formula(formula):
    element_pattern = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    return {el: int(count) if count else 1 for el, count in element_pattern}

In [23]:
print(parse_formula("Na3PS4"))

{'Na': 3, 'P': 1, 'S': 4}


In [15]:
#This function finds the correct ratio of reactants to produce one mole of the given product.
#Give the product as a dictionary of the form {"H": 2, "O": 1}
#and the reactants as a list of the form [{"H": 2, "O": 1},{"H": 2, "O": 1}].

def find_reaction_ratios(product_composition, reactant_compositions):

    # Extract all unique elements involved
    elements = set(product_composition.keys())
    for reactant in reactant_compositions:
        elements.update(reactant.keys())

    # Define unknowns (stoichiometric coefficients for reactants)
    num_reactants = len(reactant_compositions)
    coefficients = symbols(f'x1:{num_reactants + 1}')  # x1, x2, ... for reactants

    # Construct equations based on conservation of elements
    equations = []
    for element in elements:
        # Amount of element in product
        product_amount = product_composition.get(element, 0)

        # Amount of element in each reactant multiplied by its coefficient
        reactant_amounts = sum(coefficients[i] * reactant_compositions[i].get(element, 0) for i in range(num_reactants))

        # Balance equation: sum of reactants must equal product amount
        equations.append(Eq(reactant_amounts, product_amount))

    # Solve for coefficients
    solution = solve(equations, coefficients)

    # Normalize so product coefficient is 1
    min_coeff = min(solution.values())
    normalized_solution = [solution[coeff] / min_coeff for coeff in coefficients]

    return normalized_solution



In [16]:
# Example #1:
product_hashed = {"H": 2, "O": 1}  # H2O
reactants_hashed = [{"H": 2}, {"O": 2}]  # H2, O2
ratios = find_reaction_ratios(product_hashed, reactants_hashed)
print(ratios)  # Output: [1.0, 0.5] meaning 1 mol of H2 and 0.5 mol of O2

[2, 1]


In [17]:
# Example #2:
product_hashed = {"Na": 3, "P": 1,"S":4}  # Na3PS4
reactants_hashed = [{"P": 2,"S":5}, {"Na": 2,"S":1}]  # H2, O2
ratios = find_reaction_ratios(product_hashed, reactants_hashed)
print(ratios)  # Output: [1.0, 0.5] meaning 1 mol of H2 and 0.5 mol of O2

[1, 3]
